In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
pd.options.display.max_rows = 10
pd.options.display.max_columns = 500

In [2]:
import os
DIVERSITY_UTILITY = 0.25 #float(os.environ.get('DIVERSITY_UTILITY'))
FRAC_ADMIT = 0.5

###
## Load dataframe where each row is a single applicant
###

df = pd.read_csv('./df_test_large.csv')
df['key'] = df['R'].astype(str) + "_"  + df['T'].astype(str)

###
## Estimate E[Y(1)|T, A]
###

df_train = pd.read_csv('./df_train.csv')
df_stratum_utility = df_train[['R','T','Y']].groupby(['R','T']).mean().reset_index()
df_stratum_utility['stratum_utility'] = (df_stratum_utility['Y'] + DIVERSITY_UTILITY * df_stratum_utility['R']).round(2)
df_stratum_utility['key'] = df_stratum_utility['R'].astype(str) + "_" + df_stratum_utility['T'].astype(str)

df = df.merge(df_stratum_utility[['stratum_utility','key']], on='key')
df['ml_outcomes'] = df['stratum_utility']

In [3]:
FRAC_ADMIT = 0.5#df[['A']].sum()/len(df)


In [4]:
FRAC_ADMIT

0.5

In [5]:
len(df)

1000000

In [6]:
#df['ml_outcomes'] = df['R']

In [7]:
##
# R = race, T = test score, ml_outcomes = expected utility from admitting, 
# ml_outcomes_{} counterfactual utility given race 
##

df[['R','T','ml_outcomes','T_minority','T_majority','Y']].sort_values(by='ml_outcomes')



,R,T,ml_outcomes,T_minority,T_majority,Y
999934,0,11,0.00,9,11,0
999994,0,8,0.00,5,8,0
999936,0,11,0.00,11,11,0
999935,0,11,0.00,11,11,0
999933,0,11,0.00,9,11,0
...,...,...,...,...,...,...
999853,1,102,1.25,102,118,1
999854,1,102,1.25,102,117,1
999802,1,97,1.25,97,111,1
999809,1,97,1.25,97,114,1


In [8]:
#df['ml_outcomes'] = df['ml_outcomes'] + 1*df['R']

In [9]:
###
## Get total utility from admitting people in a stratum 
###

df_ = df[['R','T','Y','ml_outcomes']].groupby(['R','T','Y']).sum().reset_index()

In [10]:
###
## Get number of people in each stratum
###

df_count = df[['R','T','ml_outcomes','Y']].groupby(['R','T','Y']).count().reset_index()
df_count.columns = ['R','T','Y','Count']
df_count['N'] = df_count['Count']

In [11]:
###
## Merge summary tables to get one table with Race, Test Score, SUM(Utility), COUNT(applicants) per stratum
###

dff = df_.merge(df_count[['N']],left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False).reset_index().sort_values(by='index').reset_index()

In [12]:
# final info table
dff.sort_values(by='ml_outcomes')

,level_0,index,R,T,Y,ml_outcomes,N
0,399,0,0,8,0,0.00,1
3,398,3,0,11,0,0.00,4
5,397,5,0,12,1,0.02,1
11,396,11,0,16,1,0.03,1
9,395,9,0,15,1,0.07,1
...,...,...,...,...,...,...,...
99,4,99,0,60,1,6475.80,10793
107,3,107,0,64,1,6601.51,9853
105,2,105,0,63,1,6682.00,10280
101,1,101,0,61,1,6754.28,10894


### Setup optimization problem 

In [13]:
from ortools.linear_solver import pywraplp


In [14]:
solver = pywraplp.Solver.CreateSolver('GLOP')


In [15]:
dff

,level_0,index,R,T,Y,ml_outcomes,N
0,399,0,0,8,0,0.00,1
1,391,1,0,9,0,0.20,2
2,393,2,0,10,0,0.08,1
3,398,3,0,11,0,0.00,4
4,394,4,0,12,0,0.08,4
...,...,...,...,...,...,...,...
395,362,395,1,105,1,1.25,1
396,361,396,1,107,0,1.25,1
397,360,397,1,107,1,1.25,1
398,346,398,1,109,1,2.50,2


In [16]:
applicant_stratum = []
vars_cache = {}

# Objective: Maximize the expected utility of the admitted students
objective = solver.Objective()

# For each stratum
for ix, row in dff.iterrows():
    # probability of admission
    numvar = solver.NumVar(0.0, 1.0, str(ix))
    
    # store variable by index, and also by stratum R, T
    applicant_stratum.append(numvar)
    vars_cache[(row['R'],row['T'],row['Y'])] = numvar
    
    # Benefit of admitting people is total utility in that stratum
    objective.SetCoefficient(applicant_stratum[ix], float(row['ml_outcomes']))
objective.SetMaximization()


In [17]:
# Currently we have no constraints 
solver.NumConstraints()

0

In [18]:
# Constraint: At most K applicants
K = int(len(df)*FRAC_ADMIT)
print(K)
admit_quota = solver.Constraint(0, K)

# Total applicants cannot exceed K 
for ix, row in dff.iterrows():
    admit_quota.SetCoefficient(applicant_stratum[ix], float(row['N']))

500000


In [19]:
# Now we have one constraint
solver.NumConstraints()

1

## Add Equalized Odds Constraints

In [20]:
## Make sure that you have to add all people in Y stratum or none
## i.e. you can't add only people who pass boards and reject those who fail boards from same T, R stratum
didntexist, exists = 0, 0 

for ix, row in dff.iterrows():
    constrain_bp = solver.Constraint(0.0, 0.0)
    
    var1 = vars_cache[(row['R'],row['T'],row['Y'])]
    key2 = (row['R'],row['T'], 1-row['Y'])
    
    if key2 not in vars_cache:
        didntexist+=1
        continue
        
    var2 = vars_cache[key2]
    
    constrain_bp.SetCoefficient(var1, -1.0)
    constrain_bp.SetCoefficient(var2, 1.0)
    exists+=1

didntexist, exists

(40, 360)

In [21]:
majority_pass_boards = []
majority_fail_boards = []
minority_pass_boards = []
minority_fail_boards = []

for key in vars_cache:
    r, t, Y = key
    if Y == 1 and r==0:
        majority_pass_boards.append(key)
    elif Y == 0 and r==0:
        majority_fail_boards.append(key)
    elif Y == 1 and r==1:
        minority_pass_boards.append(key)
    elif Y == 0 and r==1:
        minority_fail_boards.append(key)

len(majority_pass_boards),len(majority_fail_boards),len(minority_pass_boards),len(minority_fail_boards)

(114, 100, 97, 89)

In [22]:
NUM_TOTALS = {}
df_totals = dff[['N','R','Y']].groupby(['R','Y']).sum().reset_index()
for ix, row in df_totals.iterrows():
    NUM_TOTALS[(row['R'],row['Y'])] = row['N']
    
N_IN_STRATAS = {}
for ix, row in dff.iterrows():
    N_IN_STRATAS[(row['R'],row['T'],row['Y'])] = row['N']

In [23]:
# Now we have one constraint
solver.NumConstraints()

401

In [24]:
#Of those who pass the boards exams
#Frac majority admitted and frac minority admitted should be the same

constrain_pass_boards = solver.Constraint(0.0, 0.0)

for key in majority_pass_boards:
    r, t, Y = key
    N_IN_STRATUM = N_IN_STRATAS[(r,t,Y)]
    N_TOTAL = NUM_TOTALS[(r,Y)]
    
    constrain_pass_boards.SetCoefficient(vars_cache[key], float(N_IN_STRATUM) / float(N_TOTAL))

for key in minority_pass_boards:
    r, t, Y = key
    N_IN_STRATUM = N_IN_STRATAS[(r,t,Y)]
    N_TOTAL = NUM_TOTALS[(r,Y)]
    
    constrain_pass_boards.SetCoefficient(vars_cache[key], -1.0 * (float(N_IN_STRATUM) / float(N_TOTAL)))


In [25]:
#Of those who fail the boards exams
#Frac majority admitted and frac minority admitted should be the same

constrain_fail_boards = solver.Constraint(0.0, 0.0)

for key in majority_fail_boards:
    r, t, Y = key
    N_IN_STRATUM = N_IN_STRATAS[(r,t,Y)]
    N_TOTAL = NUM_TOTALS[(r,Y)]
    
    constrain_fail_boards.SetCoefficient(vars_cache[key], float(N_IN_STRATUM) / float(N_TOTAL))

for key in minority_fail_boards:
    r, t, Y = key
    N_IN_STRATUM = N_IN_STRATAS[(r,t,Y)]
    N_TOTAL = NUM_TOTALS[(r,Y)]
    
    constrain_fail_boards.SetCoefficient(vars_cache[key], -1.0 * (float(N_IN_STRATUM) / float(N_TOTAL)))


## Solve linear program

In [26]:
solver.ABNORMAL

4

In [27]:
status = solver.Solve()


In [28]:
status

0

In [29]:
solver.OPTIMAL

0

In [30]:
row = []
admit = []

for i in applicant_stratum:
    row.append(int(str(i)))
    admit.append(i.solution_value())

df_decisions = pd.DataFrame({'row_id':row,'decision':admit})

In [31]:
df_decisions

,row_id,decision
0,0,0.0
1,1,1.0
2,2,1.0
3,3,0.0
4,4,0.0
...,...,...
395,395,1.0
396,396,1.0
397,397,1.0
398,398,1.0


In [32]:
dff.merge(df_decisions,left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False)

,level_0,index,R,T,Y,ml_outcomes,N,row_id,decision
103,0,103,0,62,1,6789.12,10608,103,1.0
101,1,101,0,61,1,6754.28,10894,101,1.0
105,2,105,0,63,1,6682.00,10280,105,1.0
107,3,107,0,64,1,6601.51,9853,107,1.0
99,4,99,0,60,1,6475.80,10793,99,1.0
...,...,...,...,...,...,...,...,...,...
9,395,9,0,15,1,0.07,1,9,0.0
11,396,11,0,16,1,0.03,1,11,0.0
5,397,5,0,12,1,0.02,1,5,0.0
3,398,3,0,11,0,0.00,4,3,0.0


In [33]:
xxx_ = dff.merge(df_decisions,left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False)
xxx_.sort_values(by='T')

,level_0,index,R,T,Y,ml_outcomes,N,row_id,decision
0,399,0,0,8,0,0.00,1,0,0.0
214,359,214,1,8,0,1.28,4,214,0.0
1,391,1,0,9,0,0.20,2,1,1.0
215,387,215,1,10,0,0.39,1,215,0.0
2,393,2,0,10,0,0.08,1,2,1.0
...,...,...,...,...,...,...,...,...,...
209,351,209,0,123,1,2.00,2,209,1.0
210,350,210,0,124,1,2.00,2,210,1.0
211,374,211,0,125,1,1.00,1,211,1.0
212,377,212,0,127,1,1.00,1,212,1.0


In [34]:
xxx = dff.merge(df_decisions,left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False)
xxx

,level_0,index,R,T,Y,ml_outcomes,N,row_id,decision
103,0,103,0,62,1,6789.12,10608,103,1.0
101,1,101,0,61,1,6754.28,10894,101,1.0
105,2,105,0,63,1,6682.00,10280,105,1.0
107,3,107,0,64,1,6601.51,9853,107,1.0
99,4,99,0,60,1,6475.80,10793,99,1.0
...,...,...,...,...,...,...,...,...,...
9,395,9,0,15,1,0.07,1,9,0.0
11,396,11,0,16,1,0.03,1,11,0.0
5,397,5,0,12,1,0.02,1,5,0.0
3,398,3,0,11,0,0.00,4,3,0.0


In [35]:
df['key'] = df['R'].astype(str)+'_'+df['T'].astype(str)+'_'+df['Y'].astype(str)
xxx['key'] = xxx['R'].astype(str)+'_'+xxx['T'].astype(str)+'_'+xxx['Y'].astype(str)

In [36]:
xxx

,level_0,index,R,T,Y,ml_outcomes,N,row_id,decision,key
103,0,103,0,62,1,6789.12,10608,103,1.0,0_62_1
101,1,101,0,61,1,6754.28,10894,101,1.0,0_61_1
105,2,105,0,63,1,6682.00,10280,105,1.0,0_63_1
107,3,107,0,64,1,6601.51,9853,107,1.0,0_64_1
99,4,99,0,60,1,6475.80,10793,99,1.0,0_60_1
...,...,...,...,...,...,...,...,...,...,...
9,395,9,0,15,1,0.07,1,9,0.0,0_15_1
11,396,11,0,16,1,0.03,1,11,0.0,0_16_1
5,397,5,0,12,1,0.02,1,5,0.0,0_12_1
3,398,3,0,11,0,0.00,4,3,0.0,0_11_0


In [37]:
admit_decisions = df.merge(xxx[['key','decision']],how='left',on='key')
admit_decisions['decision_random'] = pd.Series([random.random() for x in range(0,len(admit_decisions))]) < FRAC_ADMIT

FRAC_minority_POLICY = (admit_decisions['R'] * admit_decisions['decision']).sum()/admit_decisions['decision'].sum()
SUM_BP_POLICY = ( admit_decisions['Y'] * admit_decisions['decision']).sum()

FRAC_RANDOM_POLICY = (admit_decisions['R'] * admit_decisions['decision_random']).sum()/admit_decisions['decision_random'].sum()
SUM_BP_RAND_POLICY = ( admit_decisions['Y'] * admit_decisions['decision_random']).sum()

In [38]:
file = open('./lp_results.csv','a')
file.write('{}\t{}\t{}\n'.format('Counterfactual Equalized Odds',str(FRAC_minority_POLICY),str(SUM_BP_POLICY)))
file.close()

In [39]:
xxx[xxx['R']==0].sort_values(by='decision')

,level_0,index,R,T,Y,ml_outcomes,N,row_id,decision,key
40,196,40,0,31,0,332.78,2377,40,0.0,0_31_0
43,250,43,0,32,1,76.80,512,43,0.0,0_32_1
45,238,45,0,33,1,106.40,665,45,0.0,0_33_1
32,236,32,0,27,0,110.88,1008,32,0.0,0_27_0
47,228,47,0,34,1,146.54,862,47,0.0,0_34_1
...,...,...,...,...,...,...,...,...,...,...
177,222,177,0,99,1,173.63,179,177,1.0,0_99_1
146,225,146,0,84,0,158.34,174,146,1.0,0_84_0
100,46,100,0,61,0,4183.76,6748,100,1.0,0_61_0
142,211,142,0,82,0,232.20,258,142,1.0,0_82_0


In [40]:
xxx[xxx['R']==1].sort_values(by='decision')

,level_0,index,R,T,Y,ml_outcomes,N,row_id,decision,key
285,28,285,1,46,0,4797.03,9051,285,0.0,1_46_0
235,239,235,1,21,0,104.96,328,235,0.0,1_21_0
254,234,254,1,30,1,121.45,347,254,0.0,1_30_1
237,227,237,1,22,0,147.87,477,237,0.0,1_22_0
256,221,256,1,31,1,176.76,491,256,0.0,1_31_1
...,...,...,...,...,...,...,...,...,...,...
337,209,337,1,72,0,241.74,237,337,1.0,1_72_0
354,207,354,1,80,1,262.08,234,354,1.0,1_80_1
292,96,292,1,49,1,2242.86,3867,292,1.0,1_49_1
290,98,290,1,48,1,2203.60,3935,290,1.0,1_48_1


In [41]:
(xxx['N']*xxx['decision']).sum()/len(df)

0.5

In [42]:
admit_decisions[admit_decisions['Y']==1][['decision','R']].groupby(['R']).mean()

,decision
R,
0,0.676872
1,0.676872
